In [16]:
# We'll be analyzing data from these countries
countries = ["United States"]

In [17]:
import os

# In practice, you can analyze whatever data you want, as long as it's coming from data.worldbank.org
datasets = ["GDP per capita", "population growth rate", "GDP growth rate"]

# Wb gives us data in CSV format, so we'll have to do a little magic before we can use it
parsed_data = {}

def datasets(path):
    '''
    Loads all of the available datasets in a given directory.
    '''
    
    for name in os.listdir(path):
        yield csv.reader(open(os.path.join(path, name)))

In [19]:
import csv

# Now, we're actually going to load the data
data = datasets("le_data")

# And parse the data
country_data = {}

# Load entries for each country across each of the available data sets at once
for i, sources in enumerate(zip(*data)):
    # Skip any empty rows
    if len(sources[0]) <= 1:
        continue
    
    # The name of the current country is always stored in the first CSV column
    country = sources[0][0]
    
    if country in countries:
        # We'll want to collect our data from each of the provided datasets
        compiled_data = {}
        
        for j, source in enumerate(sources):
            # There's a bunch of padding by default
            data = source[4:-1]
            fixed_data = []
            
            for k, entry in enumerate(data):
                # Try to fill any gaps in the data
                if entry == "":
                    if k == 0:
                        entry = float(next(filter(lambda x : x != "", data)))
                    else:
                        entry = fixed_data[k - 1]
                else:
                    entry = float(entry)
                    
                fixed_data.append(entry)
                
            compiled_data[j] = fixed_data
            
        # Make sure we're allocating enough dates to fit the final plot of the data
        compiled_data["year"] = list(range(1960, 1960 + len(compiled_data[0])))
        
        country_data[sources[0][0]] = compiled_data

In [21]:
import matplotlib.pyplot as plt

# Make sure the data fits the diagram
plt.rcParams["figure.figsize"] = [16, 9]

_, subplots = plt.subplots(len(country_data.keys()), len(country_data[countries[0]] - 1))

for i, country in enumerate(country_data):
    data = country_data[country]
    
    # Render pairs of datasets at a time
    for j, dataset in enumerate(list(data.values())[1:-1]):
        ax = subplots[i][j]
        ax2 = ax.twinx()
        
        ax.set_xlabel("year")
        ax.set_ylabel(datasets[j])
        ax.yaxis.set_major_locator(plt.MaxNLocator(15))
        
        ax2.plot(data["year"], data[list(data.keys())[j - 1]], marker="", linewidth=2, label=country)
        ax2.yaxis.set_major_locator(plt.MaxNLocator(15))
        
plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'